In [5]:
# Numerical functions
import numpy as np
# Plotting
import matplotlib.pyplot as plt
# Units and physical constants
from astropy import units as u
from astropy import constants as c
# pandas allows us to work with spreadsheets in python
import pandas as pd
# This allows matplotlib to understand times imported from the spreadsheet
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# Calculations involving the visibility of astronomical objects
import astroplan as ap
ap.get_IERS_A_or_workaround()
# Utilities for manipulating coordinates
from astropy.time import Time
from astropy.coordinates import EarthLocation, get_body, AltAz
from astropy.coordinates import FK5, GCRS, SkyCoord, Angle

In [6]:
import astropy
print(astropy.__version__)
print(ap.__version__)

4.0
0.6


In [7]:
# Change the name in quotes to match the name of your spreadsheet
sun_stellarium = pd.read_excel('SunPositions.xlsx')

In [8]:
# Make sure the latitude and longitude match what Stellarium is assuming.
loc = EarthLocation(lat='39d57m8.39s',lon='-75d9m49.64s')
""" If we don't specify a time, the conversion assumes midnight UTC
We can make the time be noon by shifting by 17 hours (midnight to noon, and then the time
change).  """
date_noon = Time(np.array(sun_stellarium['date']), location=loc)+17*u.hr
# Just to simplify for plotting
date = date_noon.datetime64
# Ephemerides from astropy
sun_ephem_noon = get_body('sun', date_noon, loc)

In [13]:
sun_ephem_noon.ra.deg

array([281.94166822, 297.20118832, 314.99675161, 328.93154951,
       342.28528009, 355.23045488,  10.72156031,  23.55994555,
        38.5892705 ,  52.19738747,  69.31918702,  83.78116621,
       100.39505113, 114.72665905, 131.5720841 , 144.9141018 ,
       160.5493538 , 173.15487725, 187.53538221, 200.36666767,
       216.56852661, 230.61950857, 247.49105334, 262.81822642])

# Question 1

# Question 2

In [ ]:
# Define an object and an observing location for astroplan
sun_ap = ap.FixedTarget(sun_ephem_noon)
philly = ap.Observer(location=loc)

Get the transit time closest to noon
 https://astroplan.readthedocs.io/en/latest/api/astroplan.Observer.html#astroplan.Observer.target_meridian_transit_time

In [643]:
# Setting n_grid_points higher than 150 really improves the accuracy
sun_transit_ap = philly.target_meridian_transit_time(date_noon, sun_ap, n_grid_points=3000)

# Question 3

In [647]:
# This is an attempt to get the sun ephemeris, which is calculated in GCRS (effectively J2000?) into the current epoch
# I'm surprised by how hard this actually is ..
def SunEphemCurrentEpoch(sun_get_body, current_epoch):
    """ Transform the get_body('sun') RA/Dec in GCRS to a current 
    epoch RA/Dec.  current_epoch is a string 'JYYYY' where YYYY 
    is the desired decimal year """
    sun_skycoord = []
    for eph in sun_get_body:
        skycoord = SkyCoord(ra=eph.ra.deg*u.deg, dec=eph.dec.deg*u.deg, frame=GCRS, equinox='J2000.0')
        sun_skycoord.append(skycoord.transform_to(FK5(equinox=current_epoch)))
    
    sun_skycoord = SkyCoord(sun_skycoord)
    
    return sun_skycoord

# Question 4

In [ ]:
LST = date_noon.sidereal_time('apparent').hour)

# Question 5

In [572]:
sun_noon_altaz = sun_ephem_noon.transform_to(AltAz(obstime = date_noon, location = loc))